In [1]:
from D090_Ala_scan import *
init()

/home/colin/code/pyrosetta_tests/D090_Ala_scan.py:84: UserWarning: Import of 'rosetta' as a top-level module is deprecated and may be removed in 2018, import via 'pyrosetta.rosetta'.
  from rosetta.protocols.scoring import Interface


PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2020.38+release.8071b5ec488636b2cb456c6a74a0d7c19c3af497 2020-09-15T16:12:55] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release r265 2020.38+release.8071b5e 8071b5ec488636b2cb456c6a74a0d7c19c3af497 http://www.pyrosetta.org 2020-09-15T16:12:55
core.init: {0} command: PyRosetta -ex1 -ex2aro -constant_seed -database /home/colin/anaconda3/envs/PyRosetta.notebooks/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} Constant seed mode, seed=1111111 seed_offset=0 real_seed=1111111 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=1111111 RG_type=mt19937


In [2]:
# performs general scanning editing here to make parallel
def scanning_parallel(pose_copy, partners, mutant_aa = 'A',
        interface_cutoff = 8.0, output = False,
        trials = 1, trial_output = ''):
    """
    Performs "scanning" at an interface within  <pdb_filename>  between
        <partners>  by mutating relevant residues to  <mutant_aa>  and repacking
        residues within  <pack_radius>  Angstroms, further repacking all
        residues within  <interface_cutoff>  of the interface residue, scoring
        the complex and subtracting the score of a pose with the partners
        separated by 500 Angstroms.
        <trials>  scans are performed (to average results) with summaries written
        to  <trial_output>_(trial#).txt.
        Structures are exported to a PyMOL instance.

    """
    # 1. set the pose
    pose = pose_copy

    # 2. setup the docking FoldTree and other related parameters
    dock_jump = 1
    movable_jumps = Vector1([dock_jump])
#    protocols.docking.setup_foldtree(pose, partners, movable_jumps)
# Commented out by colin because of error, found that API lists this method as below in PyRosetta4
    pyrosetta.rosetta.protocols.docking.setup_foldtree(pose, partners, movable_jumps)

    # 3. create ScoreFuncions for the Interface and "ddG" calculations
    # the pose's Energies objects MUST be updated for the Interface object to
    #    work normally
    scorefxn = get_fa_scorefxn() #  create_score_function('standard')
    scorefxn(pose)    # needed for proper Interface calculation

    # setup a "ddG" ScoreFunction, custom weights
    ddG_scorefxn = ScoreFunction()
    ddG_scorefxn.set_weight(core.scoring.fa_atr, 0.44)
    ddG_scorefxn.set_weight(core.scoring.fa_rep, 0.07)
    ddG_scorefxn.set_weight(core.scoring.fa_sol, 1.0)
    ddG_scorefxn.set_weight(core.scoring.hbond_bb_sc, 0.5)
    ddG_scorefxn.set_weight(core.scoring.hbond_sc, 1.0)

    # 4. create an Interface object for the pose
    interface = Interface(dock_jump)
    interface.distance(interface_cutoff)
    interface.calculate(pose)

    # 5. create a PyMOLMover for sending output to PyMOL (optional)
#     pymover = PyMOLMover()
#     pymover.keep_history(True)    # for multiple trajectories
#     pymover.apply(pose)
#     pymover.send_energy(pose)

    # 6. perform scanning trials
    # the large number of packing operations introduces a lot of variability,
    #    for best results, perform several trials and average the results,
    #    these score changes are useful to QUALITATIVELY defining "hotspot"
    #    residues
    # this script does not use a PyJobDistributor since no PDB files are output
    
    ddG_trials = []
    
    for trial in range( trials ):
        # store the ddG values in a dictionary
        ddG_mutants = {}
        for i in range(1, pose.total_residue() + 1):
            # for residues at the interface
            if interface.is_interface(i) == True:
                # this way you can TURN OFF output by providing False arguments
                #    (such as '', the default)
                filename = ''
                if output:
                    filename = pose.pdb_info().name()[:-4] + '_' +\
                        pose.sequence()[i-1] +\
                        str(pose.pdb_info().number(i)) + '->' + mutant_aa
                # determine the interace score change upon mutation
                key = pose.sequence()[i - 1] + str(pose.pdb_info().number(i)) + mutant_aa
                ddG_mutants[key] = interface_ddG(pose, i, mutant_aa,
                    movable_jumps, ddG_scorefxn, interface_cutoff, filename )
                
        ddG_trials.append(ddG_mutants)
        
    return ddG_trials

#         # output results
#         print( '='*80 )
#         print( 'Trial', str( trial + 1 ) )
#         print( 'Mutants (PDB numbered)\t\"ddG\" (interaction dependent score change)' )
#         residues = list( ddG_mutants.keys() )  # list(...) conversion is for python3 compatbility
#         residues.sort()    # easier to read
#         display = [pose.sequence()[i - 1] +
#             str(pose.pdb_info().number(i)) + mutant_aa + '\t' +
#             str(ddG_mutants[i]) + '\n'
#             for i in residues]
#         print( ''.join(display)[:-1] )
#         print( '='*80 )

#         # write to file
#         f = open(trial_output + '_' + str(trial + 1) + '.txt' , 'w' )
#         f.writelines(display)
#         f.close()

    #### alternate output using scanning_analysis (see below), only display
    ####    mutations with "deviant" score changes
    # commented out by colin
#     print( 'Likely Hotspot Residues' )
#     for hotspot in scanning_analysis(trial_output):
#         print( hotspot )
#     print( '='*80 )

# """
# 1.  creates a copy of the pose
# 2.  sets up a specific "ddG" ScoreFunction (if no ScoreFunction is provided)
# 3.  creates a copy of the pose to mutate
# 4.  mutates a single residue using mutate_residue
# 5.  calculates the "interaction energy" ( or "binding energy")
# 6.  outputs structures (optionally):
#         -to PyMOL
#         -to a PDB file
# """

In [11]:
#test the method
pdb_filename = "./native_test.pdb"
original_pose = Pose()
pose_from_file(original_pose, pdb_filename)

#scanning_parallel(original_pose, "A_B")

poses = []
for i in range(2):
    cloned_pose = Pose()
    cloned_pose.assign(pose)
    poses.append(cloned_pose)

core.import_pose.import_pose: {0} File './native_test.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CD  on residue GLU 24
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OE1 on residue GLU 24
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OE2 on residue GLU 24
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CG  on residue GLN 73
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CD  on residue GLN 73
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OE1 on residue GLN 73
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  NE2 on residue GLN 73
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CE  on residue LYS 76
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  NZ  on residue LYS 76
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CG  on residue LYS 1

To clone a pose:
``` python
clone_pose = Pose()
clone_pose.assign(pose)
```

Looks like Dask requires some pyrosetta.distributed packages. These appear to work best when things are submitted as rosettascripts. Will have to find some better sample rosettascripts later.

Try the pure python way:
``` python
from multiprocessing import Pool
import itertools
with pyrosetta.distributed.utility.log.LoggingContext(logging.getLogger("rosetta"), level=logging.WARN):
    with Pool() as p:
        work = [
            (input_pose, i, aa, "mutation")
            for i, aa in itertools.product(range(1, len(packed_pose.to_pose(input_pose).residues) + 1), IUPACData.protein_letters)
        ]
        logging.info("mutating")
        mutations = p.starmap(mutate_residue, work)
```

In [26]:
from multiprocessing import Pool

In [34]:
%%time
all_aas = ["A", "V", "I", "L", "M", "F", "Y", "W", "S", "T", "N", "Q", "C", "U", "G", "P", "R", "H", "K", "D", "E"]
with Pool() as p:
    work = [(original_pose, "A_B", res, 8.0, False, 1) for res in all_aas]
    logging.info("doing something!!?")
    result = p.starmap(scanning_parallel, work)

core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
core.pack.task: {0} Packer task: initialize from command line()
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} built 1 rotamers at 1 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating PDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
core.pack.pack_rotamers: {0} built 1 rotamers at 1 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating PDInteracti

core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.task: {0} Packer task: initialize from command line()
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
core.pack.task: {0} Packer task: initialize from command line()
core.pack.task: {0} Packer task: initialize from command line()
core.pack.task: {0} Packer task: initialize from command line()
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
core.pack.pack_rotamers: {0} built 1 rotamers at 1 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating PDInteractionGraph
core.pack.task: {0} Packer task: initialize from command line()
core.pack.rotamer_set.RotamerSets: {0} Comple

core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.task: {0} Packer task: initialize from command line()
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.pack_rotamers: {0} built 1 rotamers at 1 positions.
core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.pack_rotamers: {0} built 98 rotamers at 10 positions.
core.pack.task: {0} Packer task: initialize from command line()
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating PDInteractionGraph
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.rotamer_set.RotamerSets: {0} 

core.pack.pack_rotamers: {0} built 1 rotamers at 1 positions.
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating PDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.pack_rotamers: {0} built 143 rotamers at 10 positions.
core.pack.pack_rotamers: {0} built 141 rotamers at 9 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interact

core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.pack_rotamers: {0} built 98 rotamers at 10 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.pack_rotamers: {0} built 141 rotamers at 9 positions.
core.pack.task: {0} Packer task: initialize from command line()
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.in

...

core.pack.pack_rotamers: {0} built 182 rotamers at 10 positions.
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.pack_rotamers: {0} built 662 rotamers at 14 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.pack_rotamers: {0} built 315 rotamers at 11 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 avail

core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.pack_rotamers: {0} built 2 rotamers at 1 positions.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating PDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} built 158 rotamers at 10 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.

core.pack.pack_rotamers: {0} built 158 rotamers at 10 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.task: {0} Packer task: initialize from command line()
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.pack_rotamers: {0} built 184 rotamers at 10 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.pack_rotamers: {0}

core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.pack_rotamers: {0} built 156 rotamers at 10 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.task: {0} Packer task: initialize from command line()
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} built 121 rotamers at 10 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} In

core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.pack_rotamers: {0} built 156 rotamers at 10 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.task: {0} Packer task: initialize from command line()
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} built 158 rotamers at 10 positions.
core.pack.pack_rotamers: {0} Requesting all available threads

Works!! Next is to parse the output into some nice data frames.